# 8 DeepAR
## b Traning on cloud
Data source: [kaggle_DailyDelhiClimate](https://www.kaggle.com/sumanthvrao/daily-climate-time-series-data?select=DailyDelhiClimateTrain.csv])

In [1]:
import time
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [ ]:
# Set the path for the train data, validation data, test data, and saved model
bucket_name = 'sagemaker-tutorial-rnd'
prefix = "DeepAR"

model_output = f"s3://{bucket_name}/{prefix}/saved_model"
train_input = f"s3://{bucket_name}/{prefix}/train/"
val_input = f"s3://{bucket_name}/{prefix}/val/"
test_input = f"s3://{bucket_name}/{prefix}/test/"

print(sagemaker.Session().boto_region_name)

In [ ]:
session = sagemaker.Session()
role = get_execution_role()
image = get_image_uri(boto3.Session().region_name, 'forecasting-deepar')

print(f"sagemaker session: {session}")
print(f"role: {role}")
print(f"image: {image}")

In [ ]:
# Set how far the prediction length is
prediction_length = 26

# Set how far in the past the network can see.
# It is recommended to set it to be the same as prediction_length
context_length = 26
job_name = "deepar-climate"

DeepAR_estimator = sagemaker.estimator.Estimator(
    image_name=image,
    role=role,
    sagemaker_session=session,
    train_instance_count=1,
    train_instance_type="ml.m5.xlarge",
    base_job_name=job_name,
    output_path=model_output
)

hyperparameters = {
    "time_freq": "w", # weekly time frequency
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length),
    "cardinality" : "auto" if with_categories else ""
}

DeepAR_estimator.set_hyperparameters(**hyperparameters)

DeepAR_estimator.fit(
    inputs = {
        "train":train_input,
        "test":val_input:
    }
)

In [ ]:
# Deploy the model
DeepAR_predictor = DeepAR_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=job_name
)